## Python で始める機器分析データの解析とケモメトリックス  
### 4.6. ベースライン補正

In [ ]:
import numpy
import pandas
from matplotlib import pyplot
from scipy import signal
filename = "data3.csv"
data = pandas.read_csv(filename, header=0, index_col=0).T
data.index = pandas.read_csv(filename, header=None, index_col=0).iloc[0].values
data.T.plot(legend=None)

(1) 1点でのベースライン補正

In [ ]:
data.iloc[0].loc[1200:1400].idxmin()  # 1200-1400 nm の範囲で最小値をとる横軸の値の検索

In [ ]:
#p.076
base = 1300  # ベースライン補正をする横軸の値
df = (data.T - data.T.loc[base]).T
df.T.plot(legend=None)

In [ ]:
#p.077
xmin, xmax = 1200, 1400  # 最小値を検索する範囲
df = data.iloc[:, (xmin < data.columns) & (data.columns < xmax)]
df = (data.T - df.T.min()).T
df.T.plot(legend=None)

(2) 2点（x1, y1), (x2, y2) を通る直線 y = a * x + b によるベースライン補正

In [ ]:
#p.077
xmin, xmax = 1200, 1400
df = data.iloc[:, (xmin < data.columns) & (data.columns < xmax)]
x1, y1 = df.idxmin(axis=1), df.min(axis=1)

xmin, xmax = 1800, 2000
df = data.iloc[:, (xmin < data.columns) & (data.columns < xmax)]
x2, y2 = df.idxmin(axis=1), df.min(axis=1)

a = (y2 - y1) / (x2 - x1)
b = y1 - a * x1
base = numpy.outer(a, data.columns) + numpy.array([b]).T

df = pandas.DataFrame(data - base, index=data.index, columns=data.columns)
df.T.plot(legend=None)

(3) Mlutiplicative Scatter Correction (MSC)

In [ ]:
#p.079
x = numpy.matrix(data)  # 測定スペクトル
xc = x - x.mean(axis=1)  # 測定スペクトルから説明変数方向の平均を引いたスペクトル
df = pandas.DataFrame(xc, index=data.index, columns=data.columns)
df.T.plot(legend=None)

In [ ]:
#p.079
r = x.mean(axis=0)  # 平均スペクトル
rc = r - r.mean()  # 平均スペクトルから説明変数方向の平均を引いたスペクトル
df = pandas.DataFrame(rc, columns=data.columns)
df.T.plot(legend=None)

In [ ]:
#p.080
x = numpy.matrix(data)
xc = x - x.mean(axis=1)
r = x.mean(axis=0)
rc = r - r.mean()
b = xc * rc.T * (rc * rc.T).I
msc = xc / b + r.mean()
df = pandas.DataFrame(msc, index=data.index, columns=data.columns)
df.T.plot(legend=None)

(4) Standard Normal Variate (SNV)

In [ ]:
#p.081
snv = ((data.T - data.T.mean()) / data.T.std()).T
snv.T.plot(legend=None)

In [ ]:
#p.082
from sklearn import preprocessing
snv = preprocessing.scale(data.T).T
pandas.DataFrame(snv, index=data.index, columns=data.columns).T.plot(legend=None)

(5) 各サンプルで正規化

In [ ]:
#p.082
xmin, xmax = 1800, 2000
df = data.iloc[:, (xmin < data.columns) & (data.columns < xmax)]
df = ((data.T - df.T.min()) / (df.T.max() - df.T.min())).T 
df.T.plot(legend=None)

(c) Shigeaki Morita